In [1]:
# Import PySpark
import findspark
findspark.init()
import pyspark # only run after findspark.init()

# Start a Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('GroupBys').getOrCreate()

# Load the data

In [2]:
df = spark.read.csv('data/sales_info.csv', inferSchema=True, header=True)
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



# Group By

In [5]:
df.groupBy('Company')

In [7]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [8]:
df.groupBy('Company').sum().show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [9]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



# Agg function

In [11]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [12]:
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [13]:
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [16]:
groupdata = df.groupBy('Company')
groupdata.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



# Using pyspark functions as aggregate functions

In [17]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [18]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [19]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [20]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [25]:
df.select([avg('Sales').alias('Average Sales'), stddev('Sales').alias('Std. Dev. Sales')]).show()

+-----------------+------------------+
|    Average Sales|   Std. Dev. Sales|
+-----------------+------------------+
|360.5833333333333|250.08742410799007|
+-----------------+------------------+



In [26]:
from pyspark.sql.functions import format_number

In [41]:
sales_avg_std = df.select([avg('Sales'), stddev('Sales')])
sales_avg_std.select([format_number('avg(Sales)', 2).alias('Avg'), format_number('stddev_samp(Sales)', 2).alias('std')]).show()

+------+------+
|   Avg|   std|
+------+------+
|360.58|250.09|
+------+------+



# Order By

In [35]:
df.orderBy(['Company', 'Sales']).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   APPL|  Linda|130.0|
|   APPL|   John|250.0|
|   APPL|  Chris|350.0|
|   APPL|   Mike|750.0|
|     FB|  Sarah|350.0|
|     FB|   Carl|870.0|
|   GOOG|Charlie|120.0|
|   GOOG|    Sam|200.0|
|   GOOG|  Frank|340.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|   MSFT|   Tina|600.0|
+-------+-------+-----+



In [37]:
df.orderBy([df['Company'].desc(), df['Sales'].desc()]).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   MSFT|   Tina|600.0|
|   MSFT|Vanessa|243.0|
|   MSFT|    Amy|124.0|
|   GOOG|  Frank|340.0|
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
+-------+-------+-----+

